# 人狼ゲームプロトタイプ

In [147]:
from dataclasses import dataclass
from typing import Optional, Final
from typing_extensions import TypeAlias
from enum import Enum, auto
import re
from logging import getLogger, StreamHandler, DEBUG
import random
import os
import time
from collections import Counter

In [141]:
def set_logger(level = DEBUG):
    logger = getLogger(__name__)
    logger.setLevel(DEBUG)
    if not logger.hasHandlers():
        handler = StreamHandler()
        handler.setLevel(DEBUG)
        logger.addHandler(handler)
    logger.propagate = False
    return logger

def log_debug(logger,text):
    if logger is not None:
        logger.debug(text)
        
        
def isInteger(value):
    """
    整数チェック
    :param value: チェック対象の文字列
    :rtype: チェック対象文字列が、全て数値の場合 True
    """
    return re.match(r"^\d+$", value) is not None


In [3]:
logger = set_logger()

In [32]:
"村人" == Roles.VILLAGER

False

In [286]:
class Roles(Enum):
    """
    役職の列挙
    今後新たな役職を追加する場合は必ずここに役職を追加する
    """
    VILLAGER = "村人"
    WEREWOLF = "人狼"
    SEER = "占い師"
    MEDIC = "騎士"
    MADMAN = "狂人"    

BASE_ROLES: Final[list[Roles]] = [
    "村人",
    "人狼"]

AVAILABLE_ROLES: Final[list[Roles]] = [
    "占い師",
    "騎士",
    "狂人"]

VILLAGER_SIDE_ROLES: Final[list[Roles]] = [
    "村人",
    "騎士",
    "占い師"]

WEREWOLF_SIDE_ROLES: Final[list[Roles]] = [
    "人狼",
    "狂人"]

PlayerName: TypeAlias = str
PlayerNames: TypeAlias = list[PlayerName]
RoleAllocation: TypeAlias = dict[Roles,int]

    
    
class GameInitializer:
    """
    ゲーム開始に必要な情報を取得する
    取得する情報
    - 使用する役職
    - 役職別の人数
    - Playerの名前
    """

    def __init__(self, logger = None):
 
        self.logger = logger
    
    def initialize(self)->tuple[RoleAllocation,PlayerNames]:
        
        self.__use_roles = self._select_using_roles()
        self.role_allocation:RoleAllocation = self._allocate_role_numbers()
        
        self.__player_num = self._get_player_num()
        self.players: Players =  self._make_players()
        
        return self.role_allocation, self.players

    def _select_using_roles(self)-> list[Roles]:
        """
        ユーザーに使用する配役を入力してもらう
        AVAILABLE_ROLESに含まれていない役職は受け付けない
        TODO
        - 利用可能なすべての役職が追加されたらループを抜ける
        - 現在の役職を表示
        """
        print("追加したい役職があればその役職名を入力してください.なければDoneと入力してください\n"\
              "使用可能な役職:")
        for i, role in enumerate(AVAILABLE_ROLES):
            print(f"- {role}")

        keep_input = True

        use_roles = BASE_ROLES.copy()
        while keep_input:
            inputted = input()

            if inputted in use_roles:
                print("既に追加済みの役職です．　その他の新たに追加したい役職を選択してください．なければDoneを入力してください")
                continue

            if inputted in AVAILABLE_ROLES:
                use_roles.append(inputted)
                print(f"使用する役職に{inputted}を追加しました．")
                print(f"その他に追加したい役職があれば役職名を入力してください.なければDoneを入力してください")

            elif inputted == "Done":
                keep_input = False

            else:
                print("無効な入力です．追加したい役職を入力するか，　Doneを入力してください")    
        self._log_debug(f"使用する配役:{use_roles}")
        return use_roles

    
    def _allocate_role_numbers(self)-> dict[Roles, int]:
        """
        村人陣営の人数が人狼陣営の人数より多くなるように配役を入力させる
        """

        unbalance = True 
        
        #妥当なバランスになるまで入力を繰り返させる
        while unbalance:
            
            #配役別の人数を入力
            role_allocation = self.__allocate_role_numbers()
            self._log_debug(f"役職別の人数:{role_allocation}")

            if self._is_valid_player_balance(role_allocation):
                unbalance = False
                
            else:
                print("人狼陣営の方が人数が多いです.")
                print("村人陣営の人数が人狼陣営の人数より多くなるように配役の人数を設定してください.")
                     
        return role_allocation
            
    
    def __allocate_role_numbers(self)->dict[Roles,int]:
        """
        役職毎の人数を入力させる,数値以外の入力は無効とする
        """
        
        role_allocation = dict()

        for role in self.__use_roles:
            print(f"役職：　{role}　の人数を入力してください")
            print(f"例:2人->[入力]:2")
            isnotint = True
            while isnotint:
                num = input()
                # TODO: 入力の妥当性検証
                if self._isInteger(num):
                    role_allocation[role] = int(num)
                    isnotint = False
                else:
                    print("無効な入力です,数値で入力してください.")
        
        return  role_allocation

    
    def _get_player_num(self):
        """playerの総数"""
        return sum([value for value in self.role_allocation.values()])
    
    
    def _make_players(self):
        players = list()
        
        for i in range(self.__player_num):
            player = Player(id_num = i)
            players.append(player)
        return players
        
    
    def _is_valid_player_balance(self, role_numbers:dict[Roles,int])->bool:
        """
        村人陣営<=人狼陣営の人数の場合,スタート直後から人狼陣営の勝利が確定しまうためチェックする必要あり
        """
        villagers_num = 0
        werewolfs_num = 0
        
        for role, num in role_numbers.items():
            self._log_debug(f"role:{role}, num:{num}")
            if role in VILLAGER_SIDE_ROLES:
                villagers_num += num

            if role in WEREWOLF_SIDE_ROLES:
                werewolfs_num += num
        self._log_debug(f"村人陣営の数:{villagers_num}")
        self._log_debug(f"人狼陣営の数:{werewolfs_num}")
        return villagers_num > werewolfs_num
        
    
    def _isInteger(self, value):
        """
        整数チェック
        :param value: チェック対象の文字列
        :rtype: チェック対象文字列が、全て数値の場合 True
        """
        return re.match(r"^\d+$", value) is not None
    
    def reset_isalive(self):
        for player in self.players:
            player.reborn()
            
    def _log_debug(self,text):
        if self.logger is not None:
            self.logger.debug(text)

In [266]:
from abc import ABC, abstractmethod
class BaseRole(ABC):
    """
    役職クラスに必要なメソッドは
    昼の行動
    夜の行動
    """
    
    @abstractmethod
    def dayaction(self):
        pass
    
    @abstractmethod
    def nightaction(self):
        pass
    
    def show_role_name(self):
        print(f"あなたの役職は{self._role_name}です")

In [305]:
class Player():

    def __init__(self,id_num):
        self.id: int = id_num

        self.name: str = self.input_name() 
        self.role: Optional[BaseRole] = None
        self.is_Alive: bool = True
        # self.is_Guardable: bool = False
        # self.is_Divinationable: bool = True
    
    
    def input_name(self):
        print(f"player{self.id}の名前を入力してください")
        name = input()
        return name

 
    def dead(self):
        self.is_Alive = False
    
    def reborn(self):
        self.is_Alive = True
#     def divined(self):
#         self.is_Divinationable = False
    
#     def change_Guardable(self):
#         self.is_Guardable = ~self.is_Guardable
    
    def set_role(self,role: Roles):
        self.role = role
        
    def dayaction(self):
        
        self.role.dayaction()
        

        
    
    def nightaction(self):
        
        self.role.nightaction()

    
Players: TypeAlias = list[Player]

class Moderator:
    
    def __init__(self,
                 role_allocation: RoleAllocation,
                 players:Players,
                 logger = None):
        self.role_allocation = role_allocation
        self.players = players
        self.aliveplayers = players
        self.deadplayers = None
        self.gamestatus = None
        self.logger = logger
        self.last_night_victim = None
        
    def assign_role(self):
        role_list = list()

        for role, num in self.role_allocation.items():
            for _ in range(num):
                role_list.append(self._set_role(Roles(role))) 
            
        if self.logger is not None:
            self.logger.debug(f"shuffle前:{role_list}")

        random.shuffle(role_list)
        
        if self.logger is not None:
            self.logger.debug(f"shuffle後:{role_list}")
            
        for player,role in zip(self.players,role_list):
            player.set_role(role)
        
  
    def _set_role(self,role: Roles)-> BaseRole:
        
        if role == Roles.VILLAGER:
            return Villager()
        
        if role == Roles.WEREWOLF:
            return Werewolf()
        
        
    def check_own_role(self):
        for player in self.players:  
            if self._is_yourself(player.name):
                player.role.show_role_name()
    
    
    def _is_yourself(self,player_name):
        """
        Player_nameと同一人物か確認をとる.
        確認が取れるまで入力を促す
        """
        notmatch = True 
        print(f"あなたは{player_name}さんですか？正しければyesと入力して下さい")
        while notmatch:
            ans = input()
            if ans == "yes":
                return True
            else:
                print("無効な入力です.")
    
    
    def _get_discussion_time(self) -> int:
        
        print("議論する時間を入力してください")
        print("例：3分 -> 入力値: 3")
        invalid_input = True
        while invalid_input:
            minute = input()        
            if isInteger(minute):
                return int(minute)
            else:
                print("無効な入力です．整数値で入力してください.")
   
    
    def set_timer(self,minute):
        second = minute * 60
        time.sleep(second)
    
    
    def select_outcast(self)-> Player:
        """
        生存しているplayerに対して， 人狼と思われるplayerの名前を入力させる
        最多投票数が複数人いる場合は最多投票者からランダムに選択
        """
        result_list = list()
        
        aliveplayer_names = [player.name for player in self.aliveplayers]
        
        for name in aliveplayer_names:
            self._is_yourself(name)
            result_list.append(self._select_outcast(aliveplayer_names))
        
        counter = Counter(result_list)
        sorted_counter = counter.most_common()

        if self._is_vote_tied(sorted_counter):
            tied_players_name = self._get_tied_players(sorted_counter)
            outcast_name = random.choice(tied_players_name)
        else:
            outcast_name = sorted_counter[0][0]
              
        outcast_index = aliveplayer_names.index(outcast_name)
        
        return self.aliveplayers[outcast_index]
    
    
    def _select_outcast(self,aliveplayer_names):
        
        print("下記playerの中から,追放する人物名を入力して下さい")
        print(aliveplayer_names)

        while True:
            selected_player = input()
            if selected_player in aliveplayer_names:
                result = list
                return selected_player
            
            else:
                print("向こうな入力で生存しているplayer名を入力して下さい")
                
                
    def _is_vote_tied(self,sorted_counter):
        if len(sorted_counter) == 1:
            return False
        
        top_vote_num = sorted_counter[0][1]
        next_vote_num = sorted_counter[1][1]
        
        if top_vote_num == next_vote_num:
            return True
        else:
            return False
        
        
    def _get_tied_players(self, sorted_counter)->list[PlayerName]:
        top_voted_num = sorted_counter[0][1]
        tied_players_name = list()
        for name, voted_num in sorted_counter:
            if voted_num == top_voted_num:
                tied_players_name.append(name)
            else:
                break
                
        return tied_players_name

    
    def update_alivalplayers(self):

        self.aliveplayers = [player for player in self.players if player.is_Alive]
        
    
    def judgment(self):
        aliveplayers_num = len(self.aliveplayers)
        werewolf_num = len([player for player in self.aliveplayers if player.role._role_name == "人狼"])
        other_roles_num = aliveplayers_num - werewolf_num
        
        if other_roles_num <= werewolf_num:
            self.gamestatus = "人狼陣営の勝利"
            
        if werewolf_num == 0:
            self.gamestatus = "村人陣営の勝利"
            
            
    def selcet_victim(self) -> Player:
        aliveplayer_names = [player.name for player in self.aliveplayers]
        target_list = list()        
        
        for player in self.aliveplayers:
            if player.role._role_name == "人狼":
                target_list.append(player.role._target_name)
                
        target_counter = Counter(target_list)
        sorted_counter = target_counter.most_common()
        
        if self._is_vote_tied(sorted_counter):
            tied_players_name = self._get_tied_players(sorted_counter)
            outcast_name = random.choice(tied_players_name)
        else:
            outcast_name = sorted_counter[0][0]
              
        target_index = aliveplayer_names.index(outcast_name)
        return self.aliveplayers[target_index]

            
    def dayaction(self):
        
        #昨晩の被害者の
        
        #議論の時間入力
        minute = self._get_discussion_time()
        #議論開始
        print("議論を開始してください.")
        self.set_timer(minute)
        
        #議論終了
        print("時間になりました議論を終了してください")
        
        self._log_player_status("select_outcast関数使用前")

        #投票
        outcast_player = self.select_outcast()
        #追放
        print(f"{outcast_player.name}が追放されます")
        
        self._log_player_status("dead関数使用前")
        
        outcast_player.dead()
        
        self._log_player_status("dead関数使用後")
                
        #勝利判定
        self.update_alivalplayers()
        
        self._log_player_status("update_alivalplayers使用後")

        self.judgment()
    
    
    def nightaction(self):
        
        #各Playerが夜の行動をとる
        for player in self.aliveplayers:
            if self._is_yourself(player.name):
                player.nightaction()
        
        #殺害する人物を決める
        victim = self.selcet_victim()
        victim.dead()
        self._log_player_status("dead使用後")
        self.last_night_victim = victim.name

        
        self.update_alivalplayers()
        self._log_player_status("update_alivalplayers使用後")

        self.judgment()
        
    def get_last_naight_victim(self):
        return self.last_night_victim
    
    
    def get_gamestatus(self):
        return self.gamestatus
    
    def _log_player_status(self,text = None):
        if self.logger is not None:
            self.logger.debug(text)
            for player in self.players:
                self.logger.debug(f"player:{player.name},status:{player.is_Alive}")

In [268]:
class Villager(BaseRole):
    """
    村人の行動
    """
    def __init__(self):
        super().__init__()
        self._role_name = "村人"
    def dayaction(self):
        pass
        
    def nightaction(self):
        pass    
    

In [269]:
class Werewolf(BaseRole):
    """
    人狼の行動
    """
    def __init__(self,logger = None):
        super().__init__()
        self.__logger = logger
        
        self._role_name = "人狼"
        
        self._target_name = None
        
    def dayaction(self):
        pass
    
    def nightaction(self):
        self.reset_target_for_kill()
        self.choose_target_for_kill()
        
    def reset_target_for_kill(self):
        self._target_name = None
    
    def choose_target_for_kill(self):
        print("殺害する人の名前を選択していください")
        name = input()
        #TODO 妥当性検証
        self._target_name = name    

In [307]:
def main():
    #ゲームの設定
    game = GameInitializer(logger=logger)
    role_allocation, players = game.initialize()

    moderator = Moderator(role_allocation,players,logger= logger)

    #Game start
    moderator.assign_role()
    moderator.check_own_role()

    day_counter = 1
    while True:
        
        print(f"{day_counter}の朝です.")
        if moderator.get_last_naight_victim() is not None:
            print(f"昨晩の被害者:{moderator.get_last_naight_victim}")
        else:
            print("昨晩の被害者はいませんでした．")
            
        #昼の行動
        moderator.dayaction()        
        if moderator.gamestatus is not None:
            print(moderator.gamestatus)
            break
        
        #夜の行動
        moderator.nightaction()
        if moderator.gamestatus is not None:
            print(moderator.gamestatus)
            break
            
        day_counter +=1

In [287]:
game = GameInitializer(logger=logger)

In [288]:
role_allocation, players = game.initialize()

追加したい役職があればその役職名を入力してください.なければDoneと入力してください
使用可能な役職:
- 占い師
- 騎士
- 狂人


 Done


使用する配役:['村人', '人狼']


役職：　村人　の人数を入力してください
例:2人->[入力]:2


 4


...

 d


player4の名前を入力してください


 e


player5の名前を入力してください


 f


In [298]:
moderator = Moderator(role_allocation,players,logger= logger)

In [302]:
moderator.assign_role()

shuffle前:[<__main__.Villager object at 0x10805fc70>, <__main__.Villager object at 0x10805f6d0>, <__main__.Villager object at 0x10805f070>, <__main__.Villager object at 0x10805f340>, <__main__.Werewolf object at 0x10805fd30>, <__main__.Werewolf object at 0x107947e80>]
shuffle後:[<__main__.Villager object at 0x10805f340>, <__main__.Villager object at 0x10805fc70>, <__main__.Werewolf object at 0x10805fd30>, <__main__.Villager object at 0x10805f070>, <__main__.Villager object at 0x10805f6d0>, <__main__.Werewolf object at 0x107947e80>]


In [303]:
moderator.dayaction()

議論する時間を入力してください
例：3分 -> 入力値: 3


 0


select_outcast関数使用前
player:a,status:True
player:c,status:True
player:b,status:True
player:d,status:True
player:e,status:True
player:f,status:True


議論を開始してください.
時間になりました議論を終了してください
あなたはaさんですか？正しければyesと入力して下さい


KeyboardInterrupt: Interrupted by user

In [293]:
moderator.aliveplayers

In [296]:
moderator.gamestatus

In [295]:
moderator.nightaction()

あなたはcさんですか？正しければyesと入力して下さい


 yes


殺害する人の名前を選択していください


 d


あなたはbさんですか？正しければyesと入力して下さい


...

 e


あなたはeさんですか？正しければyesと入力して下さい


 yes


あなたはfさんですか？正しければyesと入力して下さい


 yes


In [309]:
main()

追加したい役職があればその役職名を入力してください.なければDoneと入力してください
使用可能な役職:
- 占い師
- 騎士
- 狂人


 Done


使用する配役:['村人', '人狼']


役職：　村人　の人数を入力してください
例:2人->[入力]:2


 4


...

 yes


あなたはfさんですか？正しければyesと入力して下さい


 yes


dead使用後
player:a,status:True
player:b,status:False
player:c,status:True
player:d,status:False
player:e,status:True
player:f,status:True
update_alivalplayers使用後
player:a,status:True
player:b,status:False
player:c,status:True
player:d,status:False
player:e,status:True
player:f,status:True


村人陣営の勝利
